# Techniques Avancées de Manipulation d'Images

Ce chapitre explore des techniques plus avancées de traitement d'images en utilisant NumPy et d'autres bibliothèques spécialisées. Nous aborderons l'analyse de la distribution des pixels via des histogrammes, l'ajustement de la luminosité et du contraste, l'application de filtres de convolution pour le lissage et la détection de contours, ainsi que la composition d'images.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from skimage import data, io
from scipy.ndimage import gaussian_filter, sobel


---

## Chargement des Images d'Exemple

Nous utiliserons des images de la bibliothèque `scikit-image` pour illustrer les différentes techniques.

In [ ]:
image_couleur = data.astronaut() # Image couleur
image_gris = data.camera()    # Image en niveaux de gris

fig, axes = plt.subplots(1, 2, figsize=(10, 5))
axes[0].imshow(image_couleur)
axes[0].set_title("Image Couleur (Astronaute)")
axes[0].axis('off')
axes[1].imshow(image_gris, cmap='gray')
axes[1].set_title("Image Niveaux de Gris (Caméraman)")
axes[1].axis('off')
plt.show()

---

## 1. Histogrammes : Analyse de la Distribution des Pixels

Un histogramme d'image représente la distribution des intensités de pixels. C'est un outil essentiel pour évaluer la luminosité, le contraste et la gamme tonale d'une image. Pour le tracer, l'image 2D est d'abord "aplatie" en un tableau 1D avec la méthode `.ravel()`.

*Note : Nous utilisons l'image en niveaux de gris pour cet exemple.*

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

axes[0].imshow(image_gris, cmap='gray')
axes[0].set_title("Image en Niveaux de Gris")
axes[0].axis('off')

axes[1].hist(image_gris.ravel(), bins=256, range=(0, 255), color='gray')
axes[1].set_title("Histogramme des Intensités")
axes[1].set_xlabel("Intensité du Pixel")
axes[1].set_ylabel("Fréquence")
plt.show()

---

## 2. Ajustement de la Luminosité et du Contraste

Ces ajustements sont réalisés par des opérations arithmétiques directes sur les valeurs de pixels :

-   **Luminosité** : Ajout ou soustraction d'une constante à chaque pixel.
-   **Contraste** : Multiplication de chaque pixel par un facteur (supérieur à 1 pour augmenter, inférieur à 1 pour diminuer).

Il est essentiel d'utiliser `np.clip()` pour s'assurer que les valeurs de pixels restent dans l'intervalle valide (généralement `[0, 255]`).

In [ ]:
# Augmentation de la luminosité
image_plus_lumineuse = np.clip(image_gris.astype(int) + 50, 0, 255).astype(np.uint8)

# Augmentation du contraste
image_plus_contrastee = np.clip(image_gris.astype(float) * 1.5, 0, 255).astype(np.uint8)

fig, axes = plt.subplots(1, 3, figsize=(15, 5))
axes[0].imshow(image_gris, cmap='gray')
axes[0].set_title("Originale")
axes[0].axis('off')

axes[1].imshow(image_plus_lumineuse, cmap='gray')
axes[1].set_title("Luminosité +50")
axes[1].axis('off')

axes[2].imshow(image_plus_contrastee, cmap='gray')
axes[2].set_title("Contraste x1.5")
axes[2].axis('off')
plt.show()

---

## 3. Filtres de Convolution : Flou Gaussien

La convolution est une opération fondamentale en traitement d'images où chaque pixel de sortie est calculé comme une moyenne pondérée des pixels voisins de l'entrée. Elle est à la base de nombreux filtres.

Le **filtre gaussien** est un filtre de lissage qui réduit le bruit et adoucit les transitions. Son intensité est contrôlée par le paramètre `sigma`.

*Note : Nous utilisons la fonction `gaussian_filter` du module `scipy.ndimage`.*

In [ ]:
# Application du flou gaussien avec différents paramètres sigma
image_floue_1 = gaussian_filter(image_gris, sigma=1)
image_floue_5 = gaussian_filter(image_gris, sigma=5)

fig, axes = plt.subplots(1, 3, figsize=(15, 5))
axes[0].imshow(image_gris, cmap='gray')
axes[0].set_title("Originale")
axes[0].axis('off')

axes[1].imshow(image_floue_1, cmap='gray')
axes[1].set_title("Flou Gaussien (Sigma=1)")
axes[1].axis('off')

axes[2].imshow(image_floue_5, cmap='gray')
axes[2].set_title("Flou Gaussien (Sigma=5)")
axes[2].axis('off')
plt.show()

---

## 4. Détection de Contours (Filtre de Sobel)

La détection de contours vise à identifier les régions d'une image où les changements d'intensité sont abrupts. Le **filtre de Sobel** est un opérateur de détection de contours classique qui calcule une approximation du gradient d'intensité de l'image.

In [ ]:
contours = sobel(image_gris)

fig, axes = plt.subplots(1, 2, figsize=(10, 5))
axes[0].imshow(image_gris, cmap='gray')
axes[0].set_title("Originale")
axes[0].axis('off')

axes[1].imshow(contours, cmap='gray')
axes[1].set_title("Contour Détecté (Sobel)")
axes[1].axis('off')
plt.show()

---

## 5. Composition d'Images (Blending)

La composition d'images, ou *blending*, consiste à fusionner deux images en calculant une moyenne pondérée de leurs valeurs de pixels. Cela permet de créer des effets de transparence ou de superposition.

La formule générale est : `image_resultat = image1 * alpha + image2 * (1 - alpha)`, où `alpha` est le facteur de mélange (entre 0 et 1).

*Note : Les deux images doivent avoir les mêmes dimensions pour cette opération.*

In [ ]:
from skimage.transform import resize
from skimage.color import rgb2gray

# Chargement et préparation de deux images de même taille
image_a = data.camera()
image_b_orig = data.astronaut()
image_b_gris = rgb2gray(image_b_orig) # Convertir en niveaux de gris
image_b_redimensionnee = resize(image_b_gris, image_a.shape) # Redimensionner

# Facteur de mélange (0.5 pour un mélange équilibré)
alpha = 0.5
image_melangee = (image_a * alpha + image_b_redimensionnee * (1 - alpha))

fig, axes = plt.subplots(1, 3, figsize=(15, 5))
axes[0].imshow(image_a, cmap='gray')
axes[0].set_title("Image A (Caméraman)")
axes[0].axis('off')

axes[1].imshow(image_b_redimensionnee, cmap='gray')
axes[1].set_title("Image B (Astronaute Redimensionnée)")
axes[1].axis('off')

axes[2].imshow(image_melangee, cmap='gray')
axes[2].set_title(f"Image Mélangée (alpha={alpha})")
axes[2].axis('off')
plt.show()